In [1]:
import pymssql
import _mssql
import datetime
from decimal import Decimal

In [2]:
pymssql.__version__

'2.2.0'

In [3]:
server = 'h9ggwlagd1.database.windows.net'
user = 'steve'
password = 'KYYAtv9P'

In [4]:
import pymongo
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
db = client['tmldb']

In [5]:
def convert_dates(values):
    k,v = values
    if type(v) == datetime.date:
        return k, datetime.datetime.combine(
                v,
                datetime.datetime.min.time())
    if type(v) == Decimal:
        return k, float(v)
    else:
        return k, v
    
d = {'a': datetime.date.min, 'b': 'adadad'}

In [6]:
conn = pymssql.connect(server, user+"@"+server, password, "TMLDB")

In [31]:
c2 = conn.cursor(as_dict=True)

In [32]:
colname = 'options'
c2.execute('SELECT * FROM cqgdb.tbl'+colname)

In [33]:
collection = db[colname]

In [42]:
cnt = 0
for row in c2:
    try:
        data = dict(map(convert_dates, row.items()))
        collection.insert_one(data)    
        cnt += 1
    except TypeError:
        print('TypeError')
        print(row)
        break

print('Done {0} rows'.format(cnt))


Done 547474 rows


In [11]:
pymongo.__version__

'3.2.2'

In [ ]:
c2.rowcount

In [7]:
import sys
sys.path.append('/home/ubertrader/Dropbox/tmqrexo/tmqrv')
sys.path.append('/home/ubertrader/Dropbox/tmqrexo/tmqrv/exobuilder/tests')

In [8]:
from exobuilder.contracts.futureschain import FuturesChain
from exobuilder.contracts.futurecontract import FutureContract
from exobuilder.tests.assetindexdict import AssetIndexDicts
from datetime import datetime, date
from exobuilder.contracts.instrument import Instrument

In [9]:
assetindex = AssetIndexDicts()
symbol = 'EP'
date = datetime(2014, 1, 6, 10, 15, 0)
futures_limit = 3
instrument = Instrument(assetindex, symbol, date, futures_limit, options_limit=10)
fut_chain = FuturesChain(instrument)

In [10]:
fut_id = instrument.futures[0].dbid

In [30]:
c2 = conn.cursor(as_dict=True)
c2.execute("SELECT * FROM cqgdb.tblbardata where idcontract = '{0}'".format(fut_id))
collection = db['futures_data']

cnt = 0
for row in c2:
    try:
        data = dict(map(convert_dates, row.items()))
        collection.insert_one(data)    
        cnt += 1
    except TypeError:
        print('TypeError')
        print(row)
        break

print('Done {0} rows'.format(cnt))

Done 232092 rows


In [11]:
# Real price on ("2014-01-06 10:15:00")
instrument.futures[0]._price = 1819.5

In [12]:
opt_chain = instrument.futures[0].options[0]

In [17]:
options_dbids = []
for optpair in opt_chain:
    options_dbids.append(optpair.C.dbid)
    options_dbids.append(optpair.P.dbid)

In [23]:
tuple(options_dbids[0:2])

(11488838, 11489000)

In [28]:
c2 = conn.cursor(as_dict=True)
query = "SELECT * FROM cqgdb.tbloptiondata where idoption in {0}".format(tuple(options_dbids))
print(query)
c2.execute(query)
collection = db['options_data']

cnt = 0
for row in c2:
    try:
        data = dict(map(convert_dates, row.items()))
        collection.insert_one(data)    
        cnt += 1
    except TypeError:
        print('TypeError')
        print(row)
        break

print('Done {0} rows'.format(cnt))

SELECT * FROM cqgdb.tbloptiondata where idoption in (11488838, 11489000, 11488839, 11489001, 11488840, 11489002, 11500637, 11500722, 11488841, 11489003, 11500638, 11500723, 11488842, 11489004, 11500639, 11500724, 11488843, 11489005, 11500640, 11500725, 11488844, 11489006, 11488845, 11489007, 11488846, 11489008, 11500641, 11500726, 11488847, 11489009, 11500642, 11500727, 11488848, 11489010, 11500643, 11500728, 11488849, 11489011, 11500644, 11500729, 11488850, 11489012)
Done 4320 rows
